In [ ]:
import sys
sys.path.append(r'C:\Users\Valentino\Documents\MATLAB\PNN_wholeBrain\analysis_CTR')
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# Custom packages
import dataIO
import AbaTool
import GraphicTool as gt

# Instantiate an Atlas object from the AbaTool.py file 
# The first time you run this it will download the structures.json file from the Allen Institute server
structuresFile = r"C:\Users\Valentino\Documents\MATLAB\PNN_wholeBrain\analysis_CTR\structures.json"
A = AbaTool.Atlas(nodes=structuresFile)
DFM = AbaTool.AnatomyDataFrameManager(A)

In [ ]:
%load_ext autoreload
%autoreload 2

# Cell Intensity for all cells

In [ ]:
# ------------------------------------------------------------------------------
searchpath =  r'D:\proj_PNN-Atlas\RESULTS\colocalization_CTR'
# ------------------------------------------------------------------------------

# Load dots data from all aniamls
df = dataIO.allMiceDots(searchpath)
df = df[df['regionID']!= 0]
df = DFM.add_resolution_column(df,'regionID')

df

## Prepare data

In [ ]:
NBINS = 64

probWfa  = []
probPv  = []
intensityBins = []
mice = []
numPnn = []
numPv = []

# Select only control mice
# rawDf = df.xs('CTR', axis=0,level='treat')
rawDf = df.copy()

#extract data for each mouse and concatenate dataframes
for mouse in rawDf.index.get_level_values('mouse').unique():
    m_df = rawDf.xs(mouse, axis=0, level='mouse')                 # Select single mouse
    # Calculate histogram values for pv and wfa
    pdf_wfa, bin_edges = np.histogram(m_df['fluoMeanWfa'].dropna(), range= (0,1), density=False, bins=NBINS)
    pdf_wfa = pdf_wfa/pdf_wfa.sum()     # Normalize to total integral = 1
    pdf_pv, _ = np.histogram(m_df['fluoMeanPv'].dropna(), range= (0,1), density=False, bins=NBINS)
    pdf_pv = pdf_pv/pdf_pv.sum()        # Normalize to total integral = 1

    bins = (bin_edges[1:] + bin_edges[:-1]) / 2
    numPnn.append(m_df['fluoMeanWfa'].dropna().shape[0])
    numPv.append(m_df['fluoMeanPv'].dropna().shape[0])
    probWfa.extend(pdf_wfa)
    probPv.extend(pdf_pv)
    mice.extend([mouse]*len(bins))
    intensityBins.extend(bins)

distr = pd.DataFrame({
    'mouse':mice,
    'probWfa':probWfa,
    'probPv':probPv,
    'intBin':intensityBins,
    })

data = pd.DataFrame({
    'numPv':numPv,
    'numPnns':numPnn,
    })
# Print out informations
print("="*60)
print(f"Number of PNNs: \t{data['numPnns'].mean():.1f} \tSEM: {data['numPnns'].sem():.1f}")
print(f"Number of PVs: \t\t{data['numPv'].mean():.1f} \tSEM: {data['numPv'].sem():.1f}")
print("="*60)

# distr

## Plot Histogram for all PNNs and PV cells

In [ ]:
sns.set_style('white')

f, axs = plt.subplots(nrows=2, ncols=1, squeeze=True, figsize=(4,6))

meanColor = cm.get_cmap('PuBu')(.9)

for i, ax in enumerate(axs):
    sns.lineplot(data=distr, ax=ax,
        x='intBin',
        y= 'probWfa' if  i==0 else 'probPv',
        color=meanColor,
        errorbar='se')

    ax.xaxis.set_tick_params(labelsize=12)
    ax.yaxis.set_tick_params(labelsize=12)
    ax.set_xlabel('Single Cell Intensity (A.U.)', fontsize=16)
    ax.set_ylabel('Probability', fontsize=16)
    title = 'Perineuronal Nets Intensity' if i==0 else 'PV cells Intensity'
    ax.set_title(title, fontsize=18)

plt.tight_layout()
sns.despine()

# plt.savefig("allPNN_histogramIntensity.svg", bbox_inches="tight")

# Colocalization probability vs intensity classes

In [ ]:
colocProb = DFM.dots_to_colocProbability(
    df,
    categorized_staining='pv',
    n_bins=4
    )


colocProb

## Plot for all the brain

In [ ]:
f, ax = plt.subplots(figsize=(2.5,5.5))

gt.colocProbabilityPlot(
    colocProb,
    ax=ax,
    xlabel='PV Intensity Class',
    ylabel='Probability of having a PNN',
    txtLoc='tl')

# plt.savefig("colocProbabilityAllCells.svg", bbox_inches="tight")

## Plot for each major subdivision

In [ ]:
majorIDs = A.get_major_divisions_ids()

f, axs = plt.subplots(nrows=3, ncols=4, figsize=(10,7), squeeze=True)

for i, ax in enumerate(f.axes):
    # Select the subset dataframe for this specific brain subdivision
    thisDf = rawDf.loc[rawDf['coarseID']==majorIDs[i]]

    colocProb = DFM.dots_to_colocProbability(
        thisDf,
        categorized_staining='pv',
        n_bins=4
    )
    # colocProb = colocProb.xs('CTR', axis=0, level='treat')

    gt.colocProbabilityPlot(
        colocProb,
        ax=ax,
        xlabel='PV Intensity Class' if i==8 else None,
        ylabel='Probability of having a PNN' if i==8 else None,
        txtLoc='tl',
        fontScaling=0.8,
        ylim=[0,1],
        xticks = True if i in [8,9,10,11] else False,
        yticks = True if i in [0,4,8] else False,
        title = A.ids_to_names([majorIDs[i]])[0]
    )

# plt.savefig("colocProbability_mid.svg", bbox_inches="tight")

## All macroareas in the same plot

In [ ]:
majorIDs = A.get_major_divisions_ids()

f, ax = plt.subplots(figsize=(2.5,5.5))

for i, coarseId in enumerate(majorIDs):
    # Select the subset dataframe for this specific brain subdivision
    thisDf = rawDf.loc[rawDf['coarseID']==coarseId]

    colocProb = DFM.dots_to_colocProbability(
        thisDf,
        categorized_staining='pv',
        n_bins=4
    )
    # colocProb = colocProb.xs('CTR', axis=0, level='treat')

    gt.colocProbabilityPlot(
        colocProb,
        ax=ax,
        xlabel='PV Intensity Class' if i==11 else None,
        ylabel='Probability of having a PNN' if i==11 else None,
        txtLoc='tl',
        singleMice=False,
        avgColor=A.ids_to_colors([coarseId],color_model='rgb_norm')[0],
        fontScaling=1,
        ylim=[0,1],
        txtStat=False,
        # xticks = True if i==0 else False,
        # yticks = True if i==0 else False,
    )

ax.legend(handles = [x  for i,x in enumerate(ax.lines) if i%5==0], labels=A.ids_to_acronyms(majorIDs))

# plt.savefig("colocProbability_allToghether.svg", bbox_inches="tight")